# PPO(近端策略优化)
## 1. 从同策略到异策略
在强化学习里面，要学习的是一个智能体。如果要学习的智能体和与环境交互的智能体是相同的，称之为**同策略**。如果要学习的智能体和与环境交互的智能体不是相同的，称之为**异策略**。  
在策略梯度中，演员与环境交互搜集数据，产生很多的轨迹 τ，根据搜集到的数据按照策略梯度的公式更新策略的参数，所以策略梯度是一个同策略的算法。PPO 是策略梯度的变形，它是现在 OpenAI 默认的强化学习算法。  
$$
\bigtriangledown \bar{R}_\theta =\mathbb{E}_{\tau \sim p_\theta (\tau )}[R(\tau )\bigtriangledown \log p_\theta (\tau )] 
$$


## 2. PPO代码实现
完整代码请进项目仓库查看：https://github.com/NoneJou072/rl-notebook  
接下来具体地介绍每一个模块的实现。

### AC网络搭建

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Actor(nn.Module):
    """ 演员指策略函数，根据当前状态选择回报高的动作。因此 Actor Net 的输入是状态，输出是选择各动作的概率，
    我们希望选择概率尽可能高的动作。 """
    def __init__(self, state_dim, action_dim, hidden_dim=256) -> None:
        super().__init__()
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, action_dim)
    
    def forward(self, s):
        s = F.relu(self.fc1(s))
        s = F.relu(self.fc2(s))
        prob = F.softmax(self.fc3(s), dim=1)
        return prob
    

class Critic(nn.Module):
    """ 评论员指价值函数，用于对当前策略的值函数进行估计，即评价演员的好坏。因此 Critic Net 的输入是状态，
    输出为在当前策略下该状态上的价值, 维度为1。 """
    def __init__(self, state_dim, hidden_dim=256):
        super().__init__()
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)

    def forward(self, s):
        s = F.relu(self.fc1(s))
        s = F.relu(self.fc2(s))
        v_s = self.fc3(s)
        return v_s

### 动作采样  
对于输入的状态，为了与神经网络模型的输入要求相匹配，我们需要在第0个维度上增加一个维度。
在深度学习中，神经网络通常接受批量数据作为输入。这意味着输入数据通常具有批量维度，即一个维度用于表示输入数据的批量大小。例如，如果批量大小为32，则输入张量的形状将是(32, ...)
```python
    def sample_action(self,state):
        self.sample_count += 1
        state = torch.tensor(state, device=self.device, dtype=torch.float32).unsqueeze(dim=0)
        probs = self.actor(state)
        dist = Categorical(probs)
        action = dist.sample()
        self.log_probs = dist.log_prob(action).detach()
        return action.detach().cpu().numpy().item()

```
在给定状态的情况下，我们希望将其作为一个批量大小为1的输入传递给神经网络模型。为了满足这个要求，我们需要在输入状态的维度上增加一个维度，使其形状变为(1, ...)，其中第0个维度表示批量大小为1。

通过使用unsqueeze(dim=0)函数，我们可以在张量的第0个维度上增加一个维度，确保输入状态符合模型的输入要求。在代码中，它将状态张量的形状从(N, ...)（N是状态的维度）变为(1, N, ...)，将其转换为一个批量大小为1的张量。


### 策略更新
下面是我们要优化的目标函数：
$$
J^{\theta '}(\theta) =\mathbb{E}_{(s_t,a_t) \sim \pi _{\theta '}}[\frac{p_\theta (a_t | s_t)}{p_{\theta '}(a_t | s_t))}A^{\theta '}(s_t,a_t)] 
$$
$$
J^{\theta '}_{PPO}(\theta)  = J^{\theta '}(\theta)  - \beta KL(\theta,\theta ')
$$

---
首先，计算优势值 $A^\theta (s_t, a_t)$ ，优势值是通过对比当前策略与旧策略在相同状态下的动作价值来计算的。它用于衡量当前策略相对于旧策略在某个状态下所能获得的优势或增益。
1. 收集样本数据：使用当前策略在环境中执行一系列的动作，收集样本数据，包括状态、动作、奖励等信息。
    ```python
        # 从replay buffer中采样全部经验, 并转为tensor类型
        old_states, old_actions, old_log_probs, old_rewards, old_dones = self.memory.sample()
    ```
2. 计算动作价值：使用一个价值函数（通常是一个神经网络），根据当前策略评估每个状态下动作的价值。这可以是状态值函数（Value Function）或者是动作值函数（Q-Function）。
    ```python
        values = self.critic(old_states) 
    ```
3. 计算优势值：对于每个样本，计算其优势值。优势值的计算方式可以使用如下公式：
$A^\theta (s_t, a_t)$是用累积奖励减去基线估算出来的。  
    ```python
        advantage = discounted_rewards - values.detach()
    ```
    其中，discounted_rewards是样本收到的折扣后的累积奖励（通常使用折扣因子进行奖励的衰减），values是通过价值函数计算得到的奖励值。这里使用`detach()`是为了避免进入反向传播。

4. 计算累计奖励discounted_rewards：
    ```python
        # monte carlo estimate of state rewards
        discounted_rewards = []
        discounted_sum = 0
        for reward, done in zip(reversed(old_rewards), reversed(old_dones)):
            if done:
                discounted_sum = 0
            discounted_sum = reward + (self.gamma * discounted_sum)
            discounted_rewards.insert(0, discounted_sum)
    ```
    优势值的计算代表了当前策略相对于旧策略在某个状态下的价值提升或下降程度。如果优势值为正，则表示当前策略比旧策略在该状态下更好；如果优势值为负，则表示当前策略比旧策略在该状态下较差。

5. 归一化优势值：为了稳定训练过程，通常会对优势值进行归一化处理，使其具有零均值和单位方差。这可以通过计算优势值的均值和标准差，然后将优势值减去均值，再除以标准差来实现。
    ```python
        discounted_rewards = torch.tensor(discounted_rewards, device=self.device, dtype=torch.float32)
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-5) # 1e-5 to avoid division by zero   
    ```
通过计算优势值，PPO算法可以根据优势值的正负来进行策略更新，以鼓励良好的改进并抑制较差的改变。具体的策略更新步骤可能会使用一些近似方法，例如使用概率比率剪切（Clipping）或使用近似的概率比率损失函数来实现。这些步骤有助于确保策略更新在每次迭代中都是小步骤，并且与旧策略之间的差异控制在一个可接受的范围内。

---
其次，计算策略 $\theta$ 和 $\theta '$ 在状态 $s_t$ 下选择 $a_t$ 的条件概率，并计算它们的比值 $\frac{p_\theta (a_t | s_t)}{p_{\theta '}(a_t | s_t))}$ 。
```python
    # get new action probabilities
    probs = self.actor(old_states)
    dist = Categorical(probs)
    new_probs = dist.log_prob(old_actions)
    # compute ratio (pi_theta / pi_theta__old):
    # a/b=exp(log(a)-log(b))
    ratio = torch.exp(new_probs - old_log_probs) 
```
其中，old_log_probs 是之前从 replay buffer 中采样出来的，是我们使用示范 $\theta '$ 与环境交互时得到的对数概率。probs 是使用策略网络根据旧状态预测动作的概率，然后根据这个概率创建一个 Categorical 概率分布，再以此计算旧动作在新策略下的概率。

---
损失计算：  
1. Actor 网络损失计算   
由于传统 PPO 的 KL 散度计算较为复杂，我们使用 PPO-clip，即**近端策略优化裁剪**的方法。近端策略优化裁剪的目标函数里面没有 KL 散度，其要最大化的目标函数为  
![image](../assets/image-20220820152923-msk5jt4.png)
```python
    # compute surrogate loss
    surr1 = ratio * advantage
    surr2 = torch.clamp(ratio, 1 - self.eps_clip, 1 + self.eps_clip) * advantage
    # compute actor loss
    actor_loss = torch.min(surr1, surr2).mean()
```
2. Critic 网络损失计算  
```python
    critic_loss = F.mse_loss(discounted_rewards, values)
```

trick: Policy Entropy  
在信息论与概率统计中，熵(entropy)是表示随机变量不确定性的度量。在强化学习中，策略的熵可以表示为：
![image](../assets/screenshot-20230607-161309.png)  
一个策略的熵越大，意味着这个策略选择各个动作的概率更加“平均”。在PPO中，为了提高算法的探索能力，我们一般在actor的loss中增加一项策略熵，并乘以一个系数entropy_coef，使得在优化actor_loss的同时，让策略的熵尽可能大。一般我们设置entropy_coef=0.01。
```python
    actor_loss = -torch.min(surr1, surr2).mean() + self.entropy_coef * dist.entropy().mean()
```
